In [2]:
import re
import random
import pandas as pd

In [3]:
AGE_CHAR = [
    'novantanove',
    'novantotto',
    'novantasette',
    'novantasei',
    'novantacinque',
    'novantaquattro',
    'novantatre',
    'novantadue',
    'novantuno',
    'novanta',
    'ottantanove',
    'ottantotto',
    'ottantasette',
    'ottantasei',
    'ottantacinque',
    'ottantaquattro',
    'ottantatre',
    'ottantadue',
    'ottantuno',
    'ottanta',
    'settantanove',
    'settantotto',
    'settantasette',
    'settantasei',
    'settantacinque',
    'settantaquattro',
    'settantatre',
    'settantadue',
    'settantuno',
    'settanta',
    'sessantanove',
    'sessantotto',
    'sessantasette',
    'sessantasei',
    'sessantacinque',
    'sessantaquattro',
    'sessantatre',
    'sessantadue',
    'sessantuno',
    'sessanta',
    'cinquantanove',
    'cinquantotto',
    'cinquantasette',
    'cinquantasei',
    'cinquantacinque',
    'cinquantaquattro',
    'cinquantatre',
    'cinquantadue',
    'cinquantuno',
    'cinquanta',
    'quarantanove',
    'quarantotto',
    'quarantasette',
    'quarantasei',
    'quarantacinque',
    'quarantaquattro',
    'quarantatre',
    'quarantadue',
    'quarantuno',
    'quaranta',
    'trentanove',
    'trentotto',
    'trentasette',
    'trentasei',
    'trentacinque',
    'trentaquattro',
    'trentatre',
    'trentadue',
    'trentuno',
    'trenta',
    'ventinove',
    'ventotto',
    'ventisette',
    'ventisei',
    'venticinque',
    'ventiquattro',
    'ventitre',
    'ventidue',
    'ventuno',
    'venti',
    'diciannove',
    'diciotto',
    'diciassette',
    'sedici',
    'quindici',
    'quattordici',
    'tredici'
 ]

# remove last letter of each years_in_words entry, in order to match both
# the noun ("ventiquattro") and the adjective ("ventiquattrenne")
AGE_CHAR_SUFFIX_LONG = [year[:-1] for year in AGE_CHAR]

# keep only the shortest form as a first filter
AGE_CHAR_SUFFIX_SHORT = [
    "tredic",
    "quattordic",
    "quindic",
    "sedic",
    "diciasset",
    "diciott",
    "diciannov",
    "vent",
    "trent",
    "quarant",
    "cinquant",
    "sessant",
    "settant",
    "ottant",
    "novant",
]

AGE_DIGIT = list(range(99,12,-1))

# List of regex patterns for matching Twitter posts mentioning the age of the user
# The patterns are built using the age expressed in digits (e.g. "22" for 22)
AGE_DIGIT_PATTERNS = [
    # Matches phrases like "ho compiuto 22 anni" (I just turned 22)
    # but not "quando ho compiuto 22 anni" (when I turned 22)
    # nor "ho compiuto 22 anni di/de" (I have 22 years of)
    r"(?<!quando\s)(?<!quando)ho\s*compiuto\s*(\d{2})\s*anni(?! su)(?! più)(?! da)(?! de)(?! di)(?!de)(?!di)(?!su)(?!più)(?!da)(?! in più)(?! in meno)",
    r"\bcompio\s*(\d{2})\s*anni(?! su)(?! più)(?! da)(?! de)(?! di)(?!de)(?!di)(?!su)(?!più)(?!da)(?! in più)(?! in meno)",
    # Matches phrases like "ho 22 anni" (I am 22 years old)
    # but not "da quando/non ho 22 anni" (since I am / I am not 22 years old)
    # nor "ho 22 anni di/de" (I have 22 years of)
    # nor "se ho 22 anni" (if I am 22 years old)
    r"(?<!quando\s)(?<!quando)(?<!non\s)(?<!non)(?<!se\s)(?<!se)ho\s*(\d{2})\s*anni(?! su)(?! più)(?! da)(?! de)(?! di)(?!de)(?!di)(?!su)(?!più)(?!da)(?! in più)(?! in meno)",
    # Matches phrases like "faccio 22 anni" (I am turning 22 years old)
    # but not "faccio 22 anni di/de" (I have 22 years of)
    r"\bfaccio\s*(\d{2})\s*anni(?! che)(?! su)(?! più)(?! da)(?! de)(?! di)(?!de)(?!di)(?!su)(?!più)(?!da)(?! in più)(?! in meno)(?!che)",
    # Matches phrases like "spengo 22 candeline" (I am blowing 22 candles)
    r"\bspengo\s*(\d{2})\s*candeline",
    # Matches phrases like "il mio 22^ compleanno" (my 22nd birthday)
    r"il\s*mio\s*(\d{2})\^\s*comple(?:anno)?",
    # Matches phrases like "sono un 22enne" (I am a 22-year-old...)
    r"\bsono\s*una?\s*(\d{2})\s*enne",
    # Matches phrases like "i miei 22 anni" (my 22 years)
    # r"\bmiei\s*(\d{2})\s*anni",
]

YEAR_OF_BIRTH_PATTERNS = [
    # Matches sentences like "sono nato nel 1993/93/'93" (I was born in 1993)
    r"\bsono\s*nato\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bsono\s*nata\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono del 1993/93/'93" (I am from 1993)
    # r"sono\s*del\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono un 1993/93/'93" (I am a 1993)
    # r"sono\s*una?\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono della generazione 1993/93/'93" (I am generation 1993)
    r"sono\s*della\s*generazione\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono classe 1993/93/'93" (I am class 1993)
    r"sono\s*classe\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"sono\s*una?\s*classe\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
]

YEAR_OF_BIRTH_PATTERNS_BIO = [
    # Matches sentences like "sono nato nel 1993/93/'93" (I was born in 1993)
    r"\bsono\s*nato\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bsono\s*nata\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bnato\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bnata\s*nel\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bborn\s*in\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono del 1993/93/'93" (I am from 1993)
    r"sono\s*del\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono un 1993/93/'93" (I am a 1993)
    r"sono\s*una?\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono della generazione 1993/93/'93" (I am generation 1993)
    r"sono\s*della\s*generazione\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bgenerazione\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    # Matches sentences like "sono classe 1993/93/'93" (I am class 1993)
    r"sono\s*classe\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"sono\s*una?\s*classe\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
    r"\bclasse\s*(20[0-1][0-9]|19[0-9][0-9]|\D\d{2}\s|\D\d{2}$)",
]

def return_full_age_char_pattern(age_char):
    """
    Returns a list of regex patterns for matching Twitter posts mentioning the age of the user.
    The patterns are built using the age_char parameter, which is a string containing the
    Italian word for the age of the user (e.g. "ventidue" for 22).
    """
    age_char_patterns = [
            # Matches phrases like "ho compiuto ventidue anni" (I just turned twenty-two)
            # but not "quando ho compiuto ventidue anni" (when I turned twenty-two)
            # nor "ho compiuto ventidue anni di/de" (I have twenty-two years of)
            r"(?<!quando\s)(?<!quando)ho\s*compiuto\s*({}).*\s*anni(?! de)(?!de)(?! di)(?!di)(?! in più)(?! in meno)".format(age_char),
            r"\bcompio\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            # Matches phrases like "ho ventidue anni" (I am twenty-two years old),
            # but not "a quando/non ho ventidue anni" (since I am / I am not twenty-two years old)
            # nor "ho ventidue anni di/de" (I have twenty-two years of)
            # nor "se ho ventidue anni" (if I am twenty-two years old)
            r"(?<!quando\s)(?<!quando)(?<!non\s)(?<!non)(?<!se\s)(?<!se)ho\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)(?! in più)(?! in meno)".format(age_char),
            # Matches phrases like "faccio ventidue anni" (I am turning twenty-two years old)
            r"\bfaccio\s*({}).*\s*anni(?! de)(?! di)(?!de)(?!di)".format(age_char),
            # Matches phrases like "spengo ventidue candeline" (I am blowing twenty-two candles)
            r"\bspengo\s*({})\s*candeline".format(age_char),
            # Matches phrases like "mio ventiduesimo comple/compleanno" (my twenty-second birthday)
            r"il\s*mio\s*{}e?simo\s*comple(?:anno)?".format(age_char),
            # Matches phrases like "sono un ventiduenne" (I am twenty-two-years-old...)
            r"\bsono\s*una?\s*({})\s*e?nne".format(age_char),
            # Matches phrases like "i miei ventidue anni" (my twenty-two years)
            # r"\bmiei\s*({}).*\s*anni".format(age_char),
        ]
    return age_char_patterns

def remove_age_pattern(text):

    # search for year of birth patterns
    for pattern in YEAR_OF_BIRTH_PATTERNS:
        pattern = "(" + pattern + ")"
        newtext = re.sub(pattern, "", tweet, flags=re.IGNORECASE)
        if newtext != text:
            return newtext

    # search for year of birth patterns
    for pattern in AGE_DIGIT_PATTERNS:
        pattern = "(" + pattern + ")"
        newtext = re.sub(pattern, "", tweet, flags=re.IGNORECASE)
        if newtext != text:
            return newtext

    # check if the text contains an age expressed in characters
    if re.search(r"{}".format("|".join(AGE_CHAR_SUFFIX_SHORT)), tweet, flags=re.IGNORECASE):
        # check what age is expressed in the tweet and retrieve its index
        matching_age_char = re.findall(r"{}".format("|".join(AGE_CHAR_SUFFIX_LONG)), tweet, flags=re.IGNORECASE)[0].lower()
        matching_age_char_index = AGE_CHAR_SUFFIX_LONG.index(matching_age_char)
        # check if the age is not in a quoted text
        if not re.search(r"\".*{}.*\"".format(matching_age_char), tweet, flags=re.IGNORECASE) \
            and not re.search(r"\“.*{}.*\”".format(matching_age_char), tweet, flags=re.IGNORECASE) \
            and not re.search(r"\«.*{}.*\»".format(matching_age_char), tweet, flags=re.IGNORECASE):
            # check if also the full form of the age is present in the tweet
            if re.search(r"{}".format(AGE_CHAR[matching_age_char_index]), tweet, flags=re.IGNORECASE):
                patterns = return_full_age_char_pattern(AGE_CHAR[matching_age_char_index])
            else:
                patterns = return_full_age_char_pattern(AGE_CHAR_SUFFIX_LONG[matching_age_char_index])
            # search for age statements and retrieve age
            for pattern in patterns:
                pattern = "(" + pattern + ")"
                newtext = re.sub(pattern, "", tweet, flags=re.IGNORECASE)
                if newtext != text:
                    return newtext

    # search for year of birth patterns
    for pattern in YEAR_OF_BIRTH_PATTERNS_BIO:
        pattern = "(" + pattern + ")"
        newtext = re.sub(pattern, "", tweet, flags=re.IGNORECASE)
        if newtext != text:
            return newtext

    return text



# Example tweets with age patterns
tweets_should_match = [
    "ho compiuto 50 anni e non mi importa più niente.",
    "Ieri ho compiuto 60 anni e mi sento freschissimo.",
    "Tra un mese compio 25 anni e non vedo l'ora.",
    "Ormai ho 30 anni, damn...",
    "Finalmente faccio 18 anni!!!",
    "Cosa vuoi che ne sappia di vecchiaia, sono una 28enne...",
    "Spengo 40 candeline domani, che emozione.",
    "Grande fiesta per il mio 22^ comple.",
    "Grande fiesta per il mio 22^ compleanno.",
    "--------------------------------------------------------",
    "Ieri ho compiuto sessanta anni e mi sento freschissimo.",
    "Ieri ho compiuto sessant'anni e mi sento freschissimo.",
    "Tra un mese compio venticinque anni e non vedo l'ora.",
    "Ormai ho trent'anni, damn...",
    "Finalmente faccio diciotto anni!!!",
    "Cosa vuoi che ne sappia di vecchiaia, sono una ventottenne...",
    "Spengo quaranta candeline domani, che emozione.",
    "Grande fiesta per il mio ventiduesimo comple.",
    "Grande fiesta per il mio trentunesimo comple.",
    'Mi sento in diritto di dirlo in quanto sono un ventiduenne.',
    "HO VENTICINQUE ANNI gioventù",
]

# Example tweets with year patterns
tweets_should_match_year = [
    "Sono nato nel '52 sai?",
    "Sono nato nel 52 sai?",
    "Sono nato nel 1952 sai?",
    "Sono nato nel 2013 sai?",
    "Sono del '93",
    "Sono del 1966 sai?",
    "Sono del 2000 sai?",
    "Sono un 93",
    "Sono un 2001",
    "Sono un 93 sai?",
    "Sono un '93 sai?",
    "Sono classe '93 sai?",
    "Sono un classe 2000",
    "Ho 20 anni",
    "Ho vent'anni e non me ne frega un cavolo",
]

print("######################## Tweets that should match:")
print()
for tweet in tweets_should_match:
    print(tweet)
    print(remove_age_pattern(tweet))
    print('------------------------------')

print("######################## Tweets that should match:")
print()
for tweet in tweets_should_match_year:
    print(tweet)
    print(remove_age_pattern(tweet))
    print('------------------------------')

######################## Tweets that should match:

ho compiuto 50 anni e non mi importa più niente.
 e non mi importa più niente.
------------------------------
Ieri ho compiuto 60 anni e mi sento freschissimo.
Ieri  e mi sento freschissimo.
------------------------------
Tra un mese compio 25 anni e non vedo l'ora.
Tra un mese  e non vedo l'ora.
------------------------------
Ormai ho 30 anni, damn...
Ormai , damn...
------------------------------
Finalmente faccio 18 anni!!!
Finalmente !!!
------------------------------
Cosa vuoi che ne sappia di vecchiaia, sono una 28enne...
Cosa vuoi che ne sappia di vecchiaia, ...
------------------------------
Spengo 40 candeline domani, che emozione.
 domani, che emozione.
------------------------------
Grande fiesta per il mio 22^ comple.
Grande fiesta per .
------------------------------
Grande fiesta per il mio 22^ compleanno.
Grande fiesta per .
------------------------------
--------------------------------------------------------
--------